In [1]:
import numpy as np
import requests  
from bs4 import BeautifulSoup  
import bs4  
import re  
import traceback  
from setuptools.package_index import HREF 
from lxml import etree
class StockCode(object):
    def __init__(self):
        self.start_url = "http://quote.eastmoney.com/stocklist.html#sh"
        self.headers = {
            "User-Agent": ":Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"
        }

    def parse_url(self):
        response = requests.get(self.start_url, headers=self.headers)
        if response.status_code == 200:
            return etree.HTML(response.content)

    def get_code_list(self, response):
        node_list = response.xpath('//*[@id="quotesearch"]/ul[1]/li')
        code_list = []
        for node in node_list:
            try:
                code = re.match(r'.*?\((\d+)\)', etree.tostring(node).decode()).group(1)
                print(code)
                code_list.append(code)
            except:
                continue
        return code_list

    def run(self):
        html = self.parse_url()
        return self.get_code_list(html)

class Download_HistoryStock(object):
    import codecs

    def __init__(self, code):
        self.code = code
        self.start_url = "http://quotes.money.163.com/trade/lsjysj_" + self.code + ".html"
        print(self.start_url)
        self.headers = {
            "User-Agent": ":Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"}

    def parse_url(self):

        response = requests.get(self.start_url)
        print(response.status_code)
        if response.status_code == 200:
            return etree.HTML(response.content)
        return False

    def get_date(self, response):
        start_date = ''.join(response.xpath('//input[@name="date_start_type"]/@value')[0].split('-'))
        end_date = ''.join(response.xpath('//input[@name="date_end_type"]/@value')[0].split('-'))
        return start_date,end_date

    def download(self, start_date, end_date):
        print('Now downloading stock code:',self.code)
        
        download_url = "http://quotes.money.163.com/service/chddata.html?code=0"+self.code+"&start="+start_date+"&end="+end_date+"&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP"
        data = requests.get(download_url)
        data.encoding = ''
        title = 'D:/data/' + self.code + '.csv'
        f = open(title, 'wb')

        for chunk in data.iter_content(chunk_size=10000):
            if chunk:
                f.write(chunk)
                
        f.close()
        
        print('Download completed')

    def run(self):
        try:
            self.download('20170101', '20171231')
        except Exception as e:
            print(e)

In [ ]:
code=StockCode()
code_list = code.run()#get all stock codes
code_list_sz = [code for code in code_list if int(code)>=600000 and int(code)<=699999]
print(code_list_sz)#we only download stock codes start with '60'

In [ ]:
for code in code_list_sz:
    download = Download_HistoryStock(code)#download all stocks starting with '60'
    download.run()